# XG Boost Model

Try out the XG Boost Models with an ML Flow set-up.

In [45]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np

# Mlflow
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning
import xgboost as xgb
from xgboost import plot_importance, plot_tree
plt.style.use('fivethirtyeight')

# Model Evaluiation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## Utility Functions

In [94]:
# Function to return grouped data by year and ags5
def average_data_by_year(df):
    return df.groupby(['year', 'ags5'], as_index=False).mean()

## Read and prepare the data

In [59]:
os.listdir('data')

['20191029_Master_GRW-Fördergebiet 2014-2020 (Kreise Status).xlsx',
 'autobahn',
 'description_of_vars.pdf',
 'df_final_date_long.csv',
 'df_final_date_wide.csv',
 'df_final_stationary.csv',
 'df_final_with_dates.csv',
 'Kopie von 190320_Abgrenzung AMR nach 330. GRW-UAS_endgültig_mit Namen.xlsx',
 'monthly_toll_stats']

In [67]:
data = pd.read_csv('data\df_final_date_wide.csv')
data.shape

(11228, 40)

In [166]:
data.sort_values(by='date_new')

,ags2,ags5,date,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,number_of_companies_domestic_staff,number_of_companies_economic_services,...,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate,date_new,year
0,1,1001,2019-01-01,34.0,14.0,108.0,131.0,264.0,0.0,130.0,...,NaN,NaN,819.0,6166.0,4275.0,11452.000000,1155.0,8.2,2019-01-01,2019
9100,12,12051,2019-01-01,27.0,28.0,46.0,41.0,254.0,0.0,83.0,...,NaN,NaN,763.0,4646.0,3253.0,8568.000000,831.0,8.8,2019-01-01,2019
1988,5,5122,2019-01-01,27.0,68.0,148.0,153.0,753.0,0.0,260.0,...,NaN,NaN,1299.0,8779.0,6442.0,14608.000000,1781.0,7.5,2019-01-01,2019
9072,11,11000,2019-01-01,785.0,436.0,3944.0,10359.0,14311.0,8.0,7742.0,...,NaN,NaN,26380.0,219864.0,155838.0,496322.000000,38564.0,8.1,2019-01-01,2019
2016,5,5124,2019-01-01,60.0,88.0,330.0,331.0,1271.0,3.0,519.0,...,NaN,NaN,2075.0,28703.0,14957.0,49343.000000,3873.0,8.2,2019-01-01,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10583,15,15091,2021-04-01,135.0,175.0,185.0,53.0,720.0,0.0,396.0,...,NaN,NaN,1120.0,NaN,4540.0,8218.398818,NaN,7.1,2021-04-01,2021
9491,12,12069,2021-04-01,167.0,324.0,332.0,255.0,1354.0,1.0,782.0,...,NaN,NaN,1821.0,NaN,4983.0,6474.139495,NaN,4.3,2021-04-01,2021
587,3,3153,2021-04-01,90.0,207.0,285.0,89.0,473.0,0.0,414.0,...,NaN,NaN,970.0,NaN,4911.0,10575.351683,NaN,7.2,2021-04-01,2021
1595,3,3458,2021-04-01,78.0,508.0,240.0,118.0,661.0,0.0,492.0,...,NaN,NaN,1144.0,NaN,2693.0,5353.773648,NaN,3.6,2021-04-01,2021


In [71]:
# Check out data columns
cols = list(data.columns)
# cols.sort()
cols

['ags2',
 'ags5',
 'date',
 'number_of_companies_administration',
 'number_of_companies_agriculture',
 'number_of_companies_arts_entertainment',
 'number_of_companies_communication',
 'number_of_companies_construction',
 'number_of_companies_domestic_staff',
 'number_of_companies_economic_services',
 'number_of_companies_education',
 'number_of_companies_energy',
 'number_of_companies_extraterritorial',
 'number_of_companies_financial_and_insurance',
 'number_of_companies_health_and_social_services',
 'number_of_companies_hospitality',
 'number_of_companies_manufacturing',
 'number_of_companies_mining',
 'number_of_companies_real_estat',
 'number_of_companies_rendering_other_services',
 'number_of_companies_repair_motor_vehicles',
 'number_of_companies_technical_services',
 'number_of_companies_transport',
 'number_of_companies_unknown_sector',
 'number_of_companies_water_and_sewage',
 'number_of_company_deletions',
 'number_of_company_liquidations',
 'number_of_start_ups',
 'displayed

In [70]:
# Check null values 
data.isnull().sum()

ags2                                                                          0
ags5                                                                          0
date                                                                          0
number_of_companies_administration                                            0
number_of_companies_agriculture                                               0
number_of_companies_arts_entertainment                                        0
number_of_companies_communication                                             0
number_of_companies_construction                                              0
number_of_companies_domestic_staff                                            0
number_of_companies_economic_services                                         0
number_of_companies_education                                                 0
number_of_companies_energy                                                    0
number_of_companies_extraterritorial    

In [84]:
# Convert Date format to Datetime
data['date_new'] = pd.to_datetime(data['date'])
data[['date', 'date_new']]

,date,date_new
0,2019-01-01,2019-01-01
1,2019-02-01,2019-02-01
2,2019-03-01,2019-03-01
3,2019-04-01,2019-04-01
4,2019-05-01,2019-05-01
...,...,...
11223,2020-12-01,2020-12-01
11224,2021-01-01,2021-01-01
11225,2021-02-01,2021-02-01
11226,2021-03-01,2021-03-01


In [88]:
# Add a year variable
data['year'] = data['date_new'].dt.year
data[['date_new', 'year']]

,date_new,year
0,2019-01-01,2019
1,2019-02-01,2019
2,2019-03-01,2019
3,2019-04-01,2019
4,2019-05-01,2019
...,...,...
11223,2020-12-01,2020
11224,2021-01-01,2021
11225,2021-02-01,2021
11226,2021-03-01,2021


In [96]:
# Average unemployment by year function 
yearly_data = data.groupby(['year', 'ags5'], as_index=False).mean()
yearly_data

,year,ags5,ags2,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,number_of_companies_domestic_staff,number_of_companies_economic_services,...,employees_social_security_at_residence,employees_social_security_at_residenceemployees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate
0,2019,1001,1,28.833333,3.083333,108.750000,121.666667,204.583333,-8.25,225.833333,...,44822.000000,32532.333333,NaN,NaN,815.00,6024.666667,4174.166667,11303.250000,1105.916667,7.958333
1,2019,1002,1,114.083333,21.416667,271.083333,345.583333,556.000000,-6.75,597.916667,...,126239.000000,92193.666667,NaN,NaN,3022.50,15988.166667,10555.666667,32341.833333,2120.916667,7.591667
2,2019,1003,1,63.166667,68.166667,234.416667,202.333333,586.833333,-8.25,528.666667,...,99956.666667,80681.333333,NaN,NaN,2680.50,12466.333333,8399.500000,24632.916667,2184.416667,7.275000
3,2019,1004,1,28.750000,16.000000,85.500000,54.500000,281.250000,-8.25,213.333333,...,40941.333333,30178.000000,NaN,NaN,1226.75,4687.916667,3385.916667,8920.083333,819.500000,7.900000
4,2019,1051,1,113.250000,358.166667,213.750000,61.916667,547.750000,-8.25,344.916667,...,42763.000000,48891.666667,NaN,NaN,1054.00,5611.000000,3814.916667,10498.083333,1248.416667,5.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,2021,16073,16,136.000000,117.000000,145.500000,61.750000,649.250000,0.00,345.750000,...,NaN,NaN,NaN,NaN,673.50,NaN,3413.750000,4805.421150,NaN,6.475000
1199,2021,16074,16,134.750000,94.500000,114.250000,40.750000,560.500000,0.00,249.000000,...,NaN,NaN,NaN,NaN,763.75,NaN,2159.250000,3025.989065,NaN,4.950000
1200,2021,16075,16,133.000000,115.000000,142.500000,32.750000,577.250000,0.00,265.500000,...,NaN,NaN,NaN,NaN,834.50,NaN,2277.250000,3107.863298,NaN,5.325000
1201,2021,16076,16,124.000000,152.250000,186.250000,59.500000,789.250000,1.00,310.500000,...,NaN,NaN,NaN,NaN,675.00,NaN,2645.000000,3884.267754,NaN,5.425000


In [99]:
# Split data by years 
data2019 = yearly_data[yearly_data['year']==2019]
data2020 = yearly_data[yearly_data['year']==2020]
data2021 = yearly_data[yearly_data['year']==2021]
print(data2019.shape, data2020.shape, data2021.shape)

(401, 40) (401, 40) (401, 40)


### Define ML Problem 

Dividing the data by year and averaging over the year and then merging with the stationary data to get the predictions. 

In [116]:
# Read the stationary data
data_stat = pd.read_csv('data/df_final_stationary.csv')
data_stat.shape

(401, 178)

In [117]:
# Remove company changes data
to_be_dropped = [
    'ags2',
    'number_of_companies_administration',
    'number_of_companies_agriculture',
    'number_of_companies_arts_entertainment',
    'number_of_companies_communication',
    'number_of_companies_construction',
    'number_of_companies_domestic_staff',
    'number_of_companies_economic_services',
    'number_of_companies_education',
    'number_of_companies_energy',
    'number_of_companies_extraterritorial',
    'number_of_companies_financial_and_insurance',
    'number_of_companies_health_and_social_services',
    'number_of_companies_hospitality',
    'number_of_companies_manufacturing',
    'number_of_companies_mining',
    'number_of_companies_real_estat',
    'number_of_companies_rendering_other_services',
    'number_of_companies_repair_motor_vehicles',
    'number_of_companies_technical_services',
    'number_of_companies_transport',
    'number_of_companies_water_and_sewage'
]

data_stat.drop(to_be_dropped, axis=1, inplace=True)

## Data Prep

In [118]:
# Merge data with the stationary data
data2019_merged = pd.merge(data2019, data_stat, on='ags5')
data2020_merged = pd.merge(data2020, data_stat, on='ags5')
data2021_merged = pd.merge(data2021, data_stat, on='ags5')
data2019_merged.shape, data2020_merged.shape, data2021_merged.shape

((401, 195), (401, 195), (401, 195))

In [119]:
# Check the same columns in the data
set(data_stat.columns).intersection(set(data.columns))

{'ags5'}

In [127]:
# List of variables to be dropped from X 
to_be_dropped = [
    'ags2',
    'kreis',
    'ags5',
    'underemployment_without_short_time _work',
    'unemployed',
    'unemployment_benefit_entitled',
    'unemployment_benefit_recipients',
    'unemployment_rate', 
    'displayed_short_time_work_companies',
    'displayed_short_time_work_people',
    'employees_social_security_at_residence',
    'employees_social_security_at_residenceemployees_social_security_at_work',
    'realized_short_time_work_companies',
    'realized_short_time_work_people'    
]

In [128]:
# Set current df 
current_df = data2019_merged

# Drop variables which are not required
X = current_df.drop(to_be_dropped, axis=1)
X = pd.get_dummies(X)
y = current_df['unemployment_rate']
print(X.shape, y.shape)

(401, 188) (401,)


In [129]:
# Check X cols 
list(X.columns)

['year',
 'number_of_companies_administration',
 'number_of_companies_agriculture',
 'number_of_companies_arts_entertainment',
 'number_of_companies_communication',
 'number_of_companies_construction',
 'number_of_companies_domestic_staff',
 'number_of_companies_economic_services',
 'number_of_companies_education',
 'number_of_companies_energy',
 'number_of_companies_extraterritorial',
 'number_of_companies_financial_and_insurance',
 'number_of_companies_health_and_social_services',
 'number_of_companies_hospitality',
 'number_of_companies_manufacturing',
 'number_of_companies_mining',
 'number_of_companies_real_estat',
 'number_of_companies_rendering_other_services',
 'number_of_companies_repair_motor_vehicles',
 'number_of_companies_technical_services',
 'number_of_companies_transport',
 'number_of_companies_unknown_sector',
 'number_of_companies_water_and_sewage',
 'number_of_company_deletions',
 'number_of_company_liquidations',
 'number_of_start_ups',
 'registerd_jobs',
 'cluster'

## Set up ML Flow Training Function

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(320, 188) (81, 188) (320,) (81,)


In [164]:
def train_xgb_ts(X_train, X_test, y_train, y_test, params, run_name='xgb_model_run'):

    with mlflow.start_run(run_name=run_name):
        
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
        # print("X Train Size:", X_train.shape)
        # print("X Test Size:", X_test.shape)
        # print("y Train Size:", y_train.shape)
        # print("y Train Size:", y_test.shape)

        # print("\n")

        reg = xgb.XGBRegressor(**params)
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        
        # Calculate prediction eval metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print("Model Run Statistics")
        print(f"MSE: {mse}")
        print(f"R2 Score: {r2}")
        
        # Log params
        mlflow.log_params(params)
        mlflow.log_param('X_vars', str(list(X.columns)))
        
        # Log metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        
        # Print and save important features
        imp_features = pd.DataFrame({
            'features':X.columns, 
            'importance':model.feature_importances_
        })
        imp_features.sort_values(by='importance', ascending=False, inplace=True)
        print(imp_features.head(20))
        
        mlflow.log_param('imp_features', str(list(imp_features.head(20)['features'].values)))

        mlflow.xgboost.log_model(reg, "model")
        
        # Return the model
        return reg

### ML Experiments 

In [165]:
params_1 = {'n_estimators': 1000,
            'max_depth': 6, 
            'min_samples_split': 5,
            'learning_rate': 0.01, 
            'loss':'ls', 
            'verbosity':1}

model = train_xgb_ts(X_train, X_test, y_train, y_test, params_1, run_name='stattionary_data')

[19:26:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "loss", "min_samples_split" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Model Run Statistics
MSE: 3.929473897041176
R2 Score: -0.5670429173782054
                                       features  importance
159                                debtor_quota    0.536992
176                           metropolitan_area    0.084941
139                     cemetry_area_percentage    0.057298
168                           labor_market_type    0.036785
158                 purchasing_power_per_person    0.036078
132     industrial_/_commertial_area_percentage    0.024827
157              purchasing_power_per_household    0.010491
59                   share_of_public_tran

### Hyperparameter Tuning